In [ ]:
! pip install redis wget pandas openai python-dotenv plotly matplotlib scipy scikit-learn

In [108]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY") 
# print(openai.api_key)

In [109]:
# embedding model parameters
embedding_model = "text-embedding-ada-002" # second gen best model at the moment
embedding_encoding = "cl100k_base" # latest tokenizer for second gen models
max_tokens = 8000 # max tokens for second gen models and tokenizer above is 8191

In [110]:
# Extract rows from a csv file with specific non-empty column data
import csv
def extract_rows(csv_file, column_name, column_value):
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        rows = [row for row in reader if row[column_name] != column_value]
    return rows

In [56]:
import pandas as pd
rows_list = extract_rows('full-wmd-dataset.csv', 'symptoms', '')
df = pd.DataFrame(rows_list)

In [57]:
df.shape

(875, 7)

In [58]:
import pandas as pd
rows_list = extract_rows('full-wmd-dataset.csv', 'overview', '')
df = pd.DataFrame(rows_list)

In [59]:
df.shape

(1319, 7)

In [61]:
rows_list = extract_rows('full-wmd-dataset.csv', 'topic', '')
df = pd.DataFrame(rows_list)
df.shape

(1452, 7)

In [139]:
# Extract non-empty rows from the dataset, inspect and combine 
import pandas as pd
import numpy as np

df = pd.read_csv('full-wmd-dataset.csv')
df = df[['topic', 'overview', 'symptoms','causes', 'tests', 'treatment', 'url']]
df["combined"] = (
    "Topic: " + df.topic.fillna('') + 
    " Overview: " + df.overview.fillna('') +
    " Symptoms: " + df.symptoms.fillna('') +
    " Causes: " + df.causes.fillna('') +
    " Tests: " + df.tests.fillna('') +
    " Treatment: " + df.treatment.fillna('')
)
# df["combined"] = (
#     "Topic: " + df.topic.str.strip() + 
#     " Overview: " + df.overview.str.strip() +
#     " Symptoms: " + df.symptoms.str.strip() +
#     " Causes: " + df.causes.str.strip() +
#     " Tests: " + df.tests.str.strip() +
#     " Treatment: " + df.treatment.str.strip()
# )
# df = df.dropna() # Extract all non-empty rows
# df = df.dropna().sample(n=100) # Extract sample sized random rows

In [140]:
df.shape

(1452, 8)

In [141]:
df

,topic,overview,symptoms,causes,tests,treatment,url,combined
0,A1AT Deficiency,Alpha-1 antitrypsin deficiency is a genetic di...,You might not know you have the disease until ...,Alpha-1 antitrypsin deficiency runs in familie...,Your doctor may ask you questions such as: You...,"Although there’s no cure for AAT deficiency, y...",https://www.webmd.com/lung/copd/alpha-1-antitr...,Topic: A1AT Deficiency Overview: Alpha-1 antit...
1,AAT,Alpha-1 antitrypsin deficiency is a genetic di...,You might not know you have the disease until ...,Alpha-1 antitrypsin deficiency runs in familie...,Your doctor may ask you questions such as: You...,"Although there’s no cure for AAT deficiency, y...",https://www.webmd.com/lung/copd/alpha-1-antitr...,Topic: AAT Overview: Alpha-1 antitrypsin defic...
2,AAT Deficiency,Alpha-1 antitrypsin deficiency is a genetic di...,You might not know you have the disease until ...,Alpha-1 antitrypsin deficiency runs in familie...,Your doctor may ask you questions such as: You...,"Although there’s no cure for AAT deficiency, y...",https://www.webmd.com/lung/copd/alpha-1-antitr...,Topic: AAT Deficiency Overview: Alpha-1 antitr...
3,Abdominal Migraine,Migraines are the kind of headaches that a...,NaN,Some less common causes may include:,NaN,Some things that might help include: Headache ...,https://www.webmd.com/migraines-headaches/cycl...,Topic: Abdominal Migraine Overview: Migrai...
4,Abercrombie Syndrome,NaN,NaN,NaN,NaN,NaN,https://www.webmd.com/cancer/lymphoma/amyloido...,Topic: Abercrombie Syndrome Overview: Symptom...
...,...,...,...,...,...,...,...,...
1447,Zambusch's Disease,Oral lichen planus is a long-lasting disease t...,Symptoms can come on slowly or start all at on...,Doctors aren't sure what causes oral lichen pl...,NaN,If you feel only a slight roughness in your mo...,https://www.webmd.com/oral-health/oral-lichen-...,Topic: Zambusch's Disease Overview: Oral liche...
1448,Z-E Syndrome,Gastrinomas tend to grow slowly and are not al...,People who have Zollinger-Ellison syndrome don...,NaN,NaN,"If you have been treated for ZES, you should s...",https://www.webmd.com/digestive-disorders/zoll...,Topic: Z-E Syndrome Overview: Gastrinomas tend...
1449,ZES,Gastrinomas tend to grow slowly and are not al...,People who have Zollinger-Ellison syndrome don...,NaN,NaN,"If you have been treated for ZES, you should s...",https://www.webmd.com/digestive-disorders/zoll...,Topic: ZES Overview: Gastrinomas tend to grow ...
1450,Zika,Zika causes microcephaly in babies born to inf...,"The disease can cause fever, rash, joint pain,...","Since 2018, no local mosquito-borne virus tran...",There are two ways to test for Zika. One test ...,"There’s no treatment, but most people with sym...",https://www.webmd.com/a-to-z-guides/zika-virus...,Topic: Zika Overview: Zika causes microcephaly...


In [142]:
# Remove samples that are too long
import tiktoken

encoding = tiktoken.get_encoding(embedding_encoding)
# omit topics that are too long
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens < max_tokens]
len(df)

1452

In [143]:
# Get embeddings and save them to a csv file, slow down the requests to avoid rate limit
import time
from openai.embeddings_utils import get_embedding

def get_embedding_with_delay(text):
    time.sleep(2)
    print(time.ctime())
    return get_embedding(text, engine=embedding_model)


In [144]:
# Get embeddings and save them
from openai.embeddings_utils import get_embedding
# import time
# df["embedding"] = df.combined.sample(10).apply(lambda x: get_embedding(x, engine=embedding_model))
df["embedding"] = df.combined.apply(lambda x: get_embedding_with_delay(x))
# df["embedding"] = df.combined.sample(10).apply(lambda x: get_embedding(x, engine=embedding_model), time.sleep(1))
df.to_csv('wmd_1452_embeddings.csv')

Thu Mar  2 13:13:32 2023
Thu Mar  2 13:13:35 2023
Thu Mar  2 13:13:37 2023
Thu Mar  2 13:13:39 2023
Thu Mar  2 13:13:41 2023
Thu Mar  2 13:13:44 2023
Thu Mar  2 13:13:46 2023
Thu Mar  2 13:13:48 2023
Thu Mar  2 13:13:51 2023
Thu Mar  2 13:13:53 2023
Thu Mar  2 13:13:55 2023
Thu Mar  2 13:13:58 2023
Thu Mar  2 13:14:00 2023
Thu Mar  2 13:14:02 2023
Thu Mar  2 13:14:04 2023
Thu Mar  2 13:14:06 2023
Thu Mar  2 13:14:08 2023
Thu Mar  2 13:14:10 2023
Thu Mar  2 13:14:12 2023
Thu Mar  2 13:14:14 2023
Thu Mar  2 13:14:16 2023
Thu Mar  2 13:14:19 2023
Thu Mar  2 13:14:21 2023
Thu Mar  2 13:14:23 2023
Thu Mar  2 13:14:25 2023
Thu Mar  2 13:14:27 2023
Thu Mar  2 13:14:29 2023
Thu Mar  2 13:14:31 2023
Thu Mar  2 13:14:33 2023
Thu Mar  2 13:14:36 2023
Thu Mar  2 13:14:38 2023
Thu Mar  2 13:14:40 2023
Thu Mar  2 13:14:42 2023
Thu Mar  2 13:14:44 2023
Thu Mar  2 13:14:46 2023
Thu Mar  2 13:14:51 2023
Thu Mar  2 13:14:53 2023
Thu Mar  2 13:14:56 2023
Thu Mar  2 13:14:58 2023
Thu Mar  2 13:15:00 2023


In [145]:
df.shape

(1452, 10)

In [146]:
df

,topic,overview,symptoms,causes,tests,treatment,url,combined,n_tokens,embedding
0,A1AT Deficiency,Alpha-1 antitrypsin deficiency is a genetic di...,You might not know you have the disease until ...,Alpha-1 antitrypsin deficiency runs in familie...,Your doctor may ask you questions such as: You...,"Although there’s no cure for AAT deficiency, y...",https://www.webmd.com/lung/copd/alpha-1-antitr...,Topic: A1AT Deficiency Overview: Alpha-1 antit...,1123,"[-0.01765124686062336, 0.010525372810661793, 0..."
1,AAT,Alpha-1 antitrypsin deficiency is a genetic di...,You might not know you have the disease until ...,Alpha-1 antitrypsin deficiency runs in familie...,Your doctor may ask you questions such as: You...,"Although there’s no cure for AAT deficiency, y...",https://www.webmd.com/lung/copd/alpha-1-antitr...,Topic: AAT Overview: Alpha-1 antitrypsin defic...,1120,"[-0.0176760945469141, 0.00719116535037756, 0.0..."
2,AAT Deficiency,Alpha-1 antitrypsin deficiency is a genetic di...,You might not know you have the disease until ...,Alpha-1 antitrypsin deficiency runs in familie...,Your doctor may ask you questions such as: You...,"Although there’s no cure for AAT deficiency, y...",https://www.webmd.com/lung/copd/alpha-1-antitr...,Topic: AAT Deficiency Overview: Alpha-1 antitr...,1122,"[-0.016695192083716393, 0.008426223881542683, ..."
3,Abdominal Migraine,Migraines are the kind of headaches that a...,NaN,Some less common causes may include:,NaN,Some things that might help include: Headache ...,https://www.webmd.com/migraines-headaches/cycl...,Topic: Abdominal Migraine Overview: Migrai...,418,"[-0.006923968903720379, 0.0015569214010611176,..."
4,Abercrombie Syndrome,NaN,NaN,NaN,NaN,NaN,https://www.webmd.com/cancer/lymphoma/amyloido...,Topic: Abercrombie Syndrome Overview: Symptom...,22,"[-0.030218515545129776, -0.0016895075095817447..."
...,...,...,...,...,...,...,...,...,...,...
1447,Zambusch's Disease,Oral lichen planus is a long-lasting disease t...,Symptoms can come on slowly or start all at on...,Doctors aren't sure what causes oral lichen pl...,NaN,If you feel only a slight roughness in your mo...,https://www.webmd.com/oral-health/oral-lichen-...,Topic: Zambusch's Disease Overview: Oral liche...,326,"[0.011778105981647968, 0.014188298024237156, 0..."
1448,Z-E Syndrome,Gastrinomas tend to grow slowly and are not al...,People who have Zollinger-Ellison syndrome don...,NaN,NaN,"If you have been treated for ZES, you should s...",https://www.webmd.com/digestive-disorders/zoll...,Topic: Z-E Syndrome Overview: Gastrinomas tend...,228,"[0.0029450261499732733, -0.018525751307606697,..."
1449,ZES,Gastrinomas tend to grow slowly and are not al...,People who have Zollinger-Ellison syndrome don...,NaN,NaN,"If you have been treated for ZES, you should s...",https://www.webmd.com/digestive-disorders/zoll...,Topic: ZES Overview: Gastrinomas tend to grow ...,227,"[0.004764256998896599, -0.014849974773824215, ..."
1450,Zika,Zika causes microcephaly in babies born to inf...,"The disease can cause fever, rash, joint pain,...","Since 2018, no local mosquito-borne virus tran...",There are two ways to test for Zika. One test ...,"There’s no treatment, but most people with sym...",https://www.webmd.com/a-to-z-guides/zika-virus...,Topic: Zika Overview: Zika causes microcephaly...,802,"[-0.02849649079144001, 0.0044958991929888725, ..."


In [147]:
df.embedding

0       [-0.01765124686062336, 0.010525372810661793, 0...
1       [-0.0176760945469141, 0.00719116535037756, 0.0...
2       [-0.016695192083716393, 0.008426223881542683, ...
3       [-0.006923968903720379, 0.0015569214010611176,...
4       [-0.030218515545129776, -0.0016895075095817447...
                              ...                        
1447    [0.011778105981647968, 0.014188298024237156, 0...
1448    [0.0029450261499732733, -0.018525751307606697,...
1449    [0.004764256998896599, -0.014849974773824215, ...
1450    [-0.02849649079144001, 0.0044958991929888725, ...
1451    [0.0017186070326715708, -0.01359142642468214, ...
Name: embedding, Length: 1452, dtype: object

In [ ]:
# only if reading from csv to ensure correct type
# df["embedding"] = df.embedding.apply(eval).apply(np.array)

In [148]:
#search without redis
from openai.embeddings_utils import get_embedding, cosine_similarity

# search through the symptoms for the most similar topic
def search_symptoms(df, symptoms, n=5, pprint=True):
    symptoms_embedding = get_embedding(symptoms, engine=embedding_model)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, symptoms_embedding))
    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .combined.str.replace("Topic: ", "")
        .str.replace("Symptoms: ", "")
    )
    if pprint:
        for r in results:
            print(r[:200])
            print()
    return results

In [149]:
results = search_symptoms(df, "back pain", n=5)

Lumbar Pain Overview: We often bring on our back problems through bad habits, such as: The spine is actually a stack of 24 bones called vertebrae. A healthy spine is S-shaped when viewed from the side

Low Back Pain Exercises Overview:   Causes:  Tests:  Treatment: 

Low Back Pain Overview: The causes of back pain can be complex. Some causes of back pain include accidents, muscle strains, and sports injuries. The main symptom of cervical radiculopathy is pain that

Exercises to Reduce Low Back Pain Overview:   Causes:  Tests:  Treatment: 

Slipped Disc Overview: Back pain can sneak up on you when you least expect it. One minute you're sitting comfortably in front of the TV, and the next you try to stand up, and -- ouch! -- a sharp pain 



In [150]:
results = search_symptoms(df, "memory loss", n=5)

Monomodal Visual Amnesia Overview: The outlook for people with dissociative amnesia depends on several factors, including the person's life situation, the availability of support systems, and the indi

Agnosia, Primary Visual Overview: The outlook for people with dissociative amnesia depends on several factors, including the person's life situation, the availability of support systems, and the indiv

Agnosis, Primary Overview: The outlook for people with dissociative amnesia depends on several factors, including the person's life situation, the availability of support systems, and the individual's

Thiamine Deficiency Overview: Wernicke-Korsakoff syndrome (WKS) is one name for two conditions that often happen together -- Wernicke encephalopathy and Korsakoff syndrome. Many doctors think of them 

Gayet-Wernicke Syndrome Overview: Wernicke-Korsakoff syndrome (WKS) is one name for two conditions that often happen together -- Wernicke encephalopathy and Korsakoff syndrome. Many doctors th

In [151]:
results = search_symptoms(df, "fever and chills", n=5)

Dandy Fever Overview: Symptoms, which usually begin four to six days after infection and last for up to 10 days, may include Sometimes, symptoms are mild and can be mistaken for those of the flu or an

Scarlet Fever Overview: Scarlet fever starts out looking like a sunburn. Most often, the rash begins on the face and neck and spreads to the rest of the body. It may itch. Other signs you or your chil

Infectious Chorea Overview: A rare but potentially life-threatening disease, rheumatic fever is a complication of untreated strep throat caused by bacteria called group A streptococcus. The main sympt

Bunyavirus Infection Overview: The CDC has identified 44 countries with a risk of yellow fever transmission, many of them with tropical climates. While the actual number of yellow fever cases among U.

Rheumatic Chorea Overview: A rare but potentially life-threatening disease, rheumatic fever is a complication of untreated strep throat caused by bacteria called group A streptococcus. The mai

In [170]:
results = search_symptoms(df, "my ribs hurt when I am running", n=5)

Fractured Rib Overview: Sharp chest pain happens with a broken rib. But it’s different from a heart attack:  Causes:  Tests:  Treatment: Most broken ribs take about 6 weeks to heal. While you’re on th

Pleuritis Overview: Pleurisy is a type of chest pain. It affects a part of your body called the pleura. The pleura is a thin layer of tissue that wraps your lungs. They fit snugly within your chest, w

Lumbar Pain Overview: We often bring on our back problems through bad habits, such as: The spine is actually a stack of 24 bones called vertebrae. A healthy spine is S-shaped when viewed from the side

RICE Overview: Pain is your body’s signal that something is wrong. As soon as you’re hurt, stop your activity, and rest as much as possible for the first 2 days. Don’t try to follow the “no pain, no g

Algoneurodystrophy Overview: Doctors think the pain caused by RSD comes from problems in your sympathetic nervous system. Your sympathetic nervous system controls blood flow movements that hel

In [152]:
# start redis using the docker-compose file in the same folder
! docker compose up -d

Container redis-vector-db-1  Running


In [153]:
# connect to redis
import redis
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField
)

REDIS_HOST =  "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = "" # default for passwordless Redis

# Connect to Redis
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD
)
redis_client.ping()

True

In [154]:
# Create a search index in Redis

# Constants
VECTOR_DIM = len(df['embedding'].values[0]) # length of the vectors
VECTOR_NUMBER = len(df)                 # initial number of vectors
INDEX_NAME = "embeddings-reviews-index"   # name of the search index
PREFIX = "doc"                            # prefix for the document keys
DISTANCE_METRIC = "COSINE"                # distance metric for the vectors (ex. COSINE, IP, L2)

In [155]:
# Define RedisSearch fields for each of the columns in the dataset
topic = TextField(name="topic", weight=1.0)
overview = TextField(name="overview", weight=1.0)
symptoms = TextField(name="symptoms", weight=1.0)
url = TextField(name="url", weight=1.0)
embedding = VectorField("embedding",
                        "FLAT", {
                            "TYPE": "FLOAT32",
                            "DIM": VECTOR_DIM,
                            "DISTANCE_METRIC": DISTANCE_METRIC,
                            "INITIAL_CAP": VECTOR_NUMBER,
                        }
            )
fields = [topic, overview, symptoms, url, embedding]


In [156]:
# Check if index exists
try:
    redis_client.ft(INDEX_NAME).info()
    print("Index already exists")
except:
    # Create RediSearch Index
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
)

Index already exists


In [157]:
def index_documents(client: redis.Redis, prefix: str, documents: pd.DataFrame):
    records = documents.to_dict("records")
    for i, doc in enumerate(records):
        key = f"{prefix}:{i}"
        # key = f"{prefix}:{str(doc['id'])}"

        # create byte vectors for title and content
        embedding = np.array(doc["embedding"], dtype=np.float32).tobytes()
        # title_embedding = np.array(doc["title_vector"], dtype=np.float32).tobytes()
        # content_embedding = np.array(doc["content_vector"], dtype=np.float32).tobytes()

        # replace list of floats with byte vectors
        doc["embedding"] = embedding
        # doc["title_vector"] = title_embedding
        # doc["content_vector"] = content_embedding

        client.hset(key, mapping = doc)

In [158]:
index_documents(redis_client, PREFIX, df)
print(f"Loaded {redis_client.info()['db0']['keys']} documents in Redis search index with name: {INDEX_NAME}")

Loaded 1452 documents in Redis search index with name: embeddings-reviews-index


In [159]:
# Run a search query and return the results
from typing import List
def search_redis(
        redis_client: redis.Redis,
        user_query: str,
        index_name: str = INDEX_NAME,
        vector_field: str = "embedding",
        return_fields: list = ["topic", "overview", "symptoms", "url", "vector_score"],
        hybrid_fields = "*",
        k: int = 20,
        print_results: bool = True,
) -> List[dict]:
    """
    Search Redis for a given query and return the results.
    :param redis_client: Redis client
    :param user_query: Query string
    :param index_name: Name of the index to search in
    :param vector_field: Name of the vector field
    :param return_fields: List of fields to return
    :param hybrid_fields: List of fields to use for hybrid search
    :param k: Number of results to return
    :param print_results: Whether to print the results
    :return: List of results
    """
    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(input=user_query,
                                             model="text-embedding-ada-002",
                                             )["data"][0]['embedding']
    
    # Prepare the query
    base_query = f'{hybrid_fields}=>[KNN {k} @{vector_field} $vector AS vector_score]'

    query = (
        Query(base_query)
        .return_fields(*return_fields)
        .sort_by("vector_score")
        .paging(0, k)
        .dialect(2)
    )

    params_dict = {
        "vector": np.array(embedded_query).astype(dtype=np.float32).tobytes()
    }

    # perforrm vector search
    results = redis_client.ft(index_name).search(query, params_dict)

    # Print the results
    if print_results:
        for i, result in enumerate(results.docs):
            print(f"Rank: {i}")
            print(f"Topic: {result.topic}")
            print(f"Overview: {result.overview}")
            print(f"Symptoms: {result.symptoms}")
            print(f"URL: {result.url}")
            score = 1 - float(result.vector_score)
            print(f"Score: {round(score, 3)})")
            print()

    return results.docs

In [160]:
# Search redis
results = search_redis(redis_client, 'back pain', k=10)

Rank: 0
Topic: Lumbar Pain
Overview: We often bring on our back problems through bad habits, such as: The spine is actually a stack of 24 bones called vertebrae. A healthy spine is S-shaped when viewed from the side. It curves back at your shoulders and inward at your neck and small of your back. It houses and protects your spinal cord, the network of nerves that transmit feeling and control movement throughout your entire body.  One of the more common types of back pain comes from straining the bands of muscles surrounding the spine. It happens most often in the curve of the low back and the base of the neck. These areas support more weight than your upper and mid back, which are less prone to trouble. Injuries from contact sports, accidents, and falls can cause problems ranging from minor muscle strains, to herniated disks, to fractures that damage the spinal column or cord. Stabbing low back pain could be from muscle spasms, when your muscles seize up and don't relax, like a cramp. 

In [163]:
# Search redis
results = search_redis(redis_client, 'fever and chills', k=10)

Rank: 0
Topic: Dandy Fever
Overview: Symptoms, which usually begin four to six days after infection and last for up to 10 days, may include Sometimes, symptoms are mild and can be mistaken for those of the flu or another viral infection. Younger children and people who have never had the infection before tend to have milder cases than older children and adults. However, serious problems can develop. These include dengue hemorrhagic fever, a rare complication characterized by high fever, damage to lymph and blood vessels, bleeding from the nose and gums, enlargement of the liver, and failure of the circulatory system. The symptoms may progress to massive bleeding, shock, and death. This is called dengue shock syndrome (DSS). People with weakened immune systems as well as those with a second or subsequent dengue infection are believed to be at greater risk for developing dengue hemorrhagic fever.
Symptoms: Symptoms, which usually begin four to six days after infection and last for up to 

In [164]:
# Search redis
results = search_redis(redis_client, 'memory loss', k=10)

Rank: 0
Topic: Monomodal Visual Amnesia
Overview: The outlook for people with dissociative amnesia depends on several factors, including the person's life situation, the availability of support systems, and the individual's response to treatment. For most people with dissociative amnesia, memory returns with time, making the overall outlook very good. 
Symptoms: The primary symptom of dissociative amnesia is the sudden inability to remember past experiences or personal information. Some people with this disorder also might appear confused and suffer from depression and/or anxiety, or psychiatri disorders. 
URL: https://www.webmd.com/mental-health/dissociative-amnesia
Score: 0.831)

Rank: 1
Topic: Agnosia, Primary Visual
Overview: The outlook for people with dissociative amnesia depends on several factors, including the person's life situation, the availability of support systems, and the individual's response to treatment. For most people with dissociative amnesia, memory returns with 

In [168]:
# Search redis
results = search_redis(redis_client, 'purple toes and cold shivers and light headedness', k=10)

Rank: 0
Topic: CMTC
Overview: The main sign is the blue or purple marbled pattern on the skin. It looks a lot like what happens to a baby’s skin when they’re cold, but it’s more defined and doesn’t go away. Most children get it on the legs, but it can happen on the arms, torso, or very rarely, the face and scalp. The pattern may get larger when your child moves, cries, or gets cold.  It doesn’t happen often, but CMTC can also cause:
Symptoms: The main sign is the blue or purple marbled pattern on the skin. It looks a lot like what happens to a baby’s skin when they’re cold, but it’s more defined and doesn’t go away. Most children get it on the legs, but it can happen on the arms, torso, or very rarely, the face and scalp. The pattern may get larger when your child moves, cries, or gets cold.  It doesn’t happen often, but CMTC can also cause:
URL: https://www.webmd.com/skin-problems-and-treatments/cutis-marmorata-telangiectatica-congenita
Score: 0.801)

Rank: 1
Topic: Van Lohuizen Syndr

In [169]:
# Search redis
results = search_redis(redis_client, 'my ribs hurt when I am running', k=10)

Rank: 0
Topic: Fractured Rib
Overview: Sharp chest pain happens with a broken rib. But it’s different from a heart attack:
Symptoms: nan
URL: https://www.webmd.com/a-to-z-guides/do-i-have-a-broken-rib
Score: 0.819)

Rank: 1
Topic: Pleuritis
Overview: Pleurisy is a type of chest pain. It affects a part of your body called the pleura. The pleura is a thin layer of tissue that wraps your lungs. They fit snugly within your chest, which is lined with another thin layer of pleura. These layers keep your bare lungs from rubbing against the wall of your chest cavity every time you breathe in. There’s a bit of fluid within the narrow space between the two layers of pleura to keep everything moving smoothly. When you’re healthy, you never notice your pleura at work. But if your pleura has a problem, you’ll feel it. When the pleurae are swollen and inflamed, they rub against each other in a very painful way each time your lungs expand. When you inhale deeply, cough, sneeze, or laugh, you’ll proba

In [172]:
results = search_redis(redis_client, 'I have a high fever and shivers', k=10)

Rank: 0
Topic: Dandy Fever
Overview: Symptoms, which usually begin four to six days after infection and last for up to 10 days, may include Sometimes, symptoms are mild and can be mistaken for those of the flu or another viral infection. Younger children and people who have never had the infection before tend to have milder cases than older children and adults. However, serious problems can develop. These include dengue hemorrhagic fever, a rare complication characterized by high fever, damage to lymph and blood vessels, bleeding from the nose and gums, enlargement of the liver, and failure of the circulatory system. The symptoms may progress to massive bleeding, shock, and death. This is called dengue shock syndrome (DSS). People with weakened immune systems as well as those with a second or subsequent dengue infection are believed to be at greater risk for developing dengue hemorrhagic fever.
Symptoms: Symptoms, which usually begin four to six days after infection and last for up to 

AttributeError: 'Document' object has no attribute 'topic'